In [1]:
"""
FA 590 Statistical Learning in Finance - Final Project
VERSION: Linear + Tree-Based + Neural Network Models
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
import time

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from scipy import stats

# TensorFlow for Neural Network
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# Set style for professional charts
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print("="*80)
print("FA 590 - Stock Return Prediction Project - FULL VERSION")
print("="*80)

# ============================================================================
# 1. DATA LOADING
# ============================================================================
print("\n[STEP 1] Loading Data...")

# UPDATE THIS PATH
DATA_PATH = '/Users/harshil/Desktop/Statistical Learning Final Project/return_predictability_data.csv'

# Get directory for saving outputs
OUTPUT_DIR = os.path.dirname(DATA_PATH)
CHARTS_DIR = os.path.join(OUTPUT_DIR, 'charts')
os.makedirs(CHARTS_DIR, exist_ok=True)

print(f"Data location: {DATA_PATH}")
print(f"Outputs will be saved to: {OUTPUT_DIR}")
print(f"Charts will be saved to: {CHARTS_DIR}")

df = pd.read_csv(DATA_PATH)

print(f"\nDataset Shape: {df.shape}")
print(f"Dataset Size: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB in memory")
print(f"Date Range: {df['DATE'].min()} to {df['DATE'].max()}")
print(f"Number of unique stocks: {df['permno'].nunique()}")
print(f"Number of unique dates: {df['DATE'].nunique()}")

# ============================================================================
# 2. DATA PREPROCESSING
# ============================================================================
print("\n[STEP 2] Data Preprocessing...")

ID_COLS = ['permno', 'DATE']
if 'name' in df.columns:
    ID_COLS.append('name')

TARGET = 'RET'
FEATURE_COLS = [col for col in df.columns if col not in ID_COLS + [TARGET]]

print(f"Number of features: {len(FEATURE_COLS)}")

# Convert to numeric
print("\n[Converting data types...]")
df[TARGET] = pd.to_numeric(df[TARGET], errors='coerce')

# Handle missing values
df = df.sort_values(['permno', 'DATE'])
df[FEATURE_COLS] = df.groupby('permno')[FEATURE_COLS].ffill()

for col in FEATURE_COLS:
    if df[col].isnull().sum() > 0:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col].fillna(df[col].median(), inplace=True)

df = df.dropna(subset=[TARGET])
print(f"Dataset shape after preprocessing: {df.shape}")

# Handle sic2
if 'sic2' in df.columns and 'sic2' in FEATURE_COLS:
    print("\n[Encoding categorical variable: sic2]")
    sic2_dummies = pd.get_dummies(df['sic2'], prefix='sic2', drop_first=True)
    df = pd.concat([df, sic2_dummies], axis=1)
    df = df.drop('sic2', axis=1)
    FEATURE_COLS = [col for col in df.columns if col not in ID_COLS + [TARGET]]
    print(f"Features after encoding: {len(FEATURE_COLS)}")

# ============================================================================
# 3. EXPLORATORY DATA ANALYSIS WITH CHARTS
# ============================================================================
print("\n[STEP 3] Exploratory Data Analysis & Creating Charts...")

# CHART 1: Target Distribution (Multi-panel)
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].hist(df[TARGET], bins=100, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_xlabel('Monthly Return (RET)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Full Distribution of Stock Returns')
axes[0, 0].axvline(df[TARGET].mean(), color='red', linestyle='--', label=f'Mean: {df[TARGET].mean():.4f}')
axes[0, 0].legend()

axes[0, 1].hist(df[TARGET], bins=100, edgecolor='black', alpha=0.7, color='darkgreen')
axes[0, 1].set_xlabel('Monthly Return (RET)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Returns Distribution (Zoomed: -50% to +50%)')
axes[0, 1].set_xlim(-0.5, 0.5)

axes[1, 0].boxplot(df[TARGET], vert=True)
axes[1, 0].set_ylabel('Monthly Return')
axes[1, 0].set_title('Boxplot of Stock Returns')
axes[1, 0].grid(True, alpha=0.3)

# QQ plot
stats.probplot(df[TARGET], dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot: Returns vs Normal Distribution')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '01_target_distribution.png'), bbox_inches='tight')
print("✓ Chart 1: Target Distribution saved")
plt.close()

# CHART 2: Returns Over Time
print("\n[Creating time series chart...]")
monthly_avg = df.groupby('DATE')[TARGET].mean().reset_index()
monthly_avg['DATE'] = pd.to_datetime(monthly_avg['DATE'])

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(monthly_avg['DATE'], monthly_avg[TARGET], linewidth=1.5, color='darkblue')
ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax.fill_between(monthly_avg['DATE'], monthly_avg[TARGET], alpha=0.3, color='skyblue')
ax.set_xlabel('Date')
ax.set_ylabel('Average Monthly Return')
ax.set_title('Average Stock Returns Over Time (2009-2021)')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '02_returns_over_time.png'), bbox_inches='tight')
print("✓ Chart 2: Returns Over Time saved")
plt.close()

# CHART 3: Top Feature Correlations
print("\n[Calculating correlations...]")
numeric_cols = df[FEATURE_COLS].select_dtypes(include=[np.number]).columns[:100]
correlations = df[numeric_cols].corrwith(df[TARGET]).abs().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 8))
top_20 = correlations.head(20)
ax.barh(range(len(top_20)), top_20.values, color='teal')
ax.set_yticks(range(len(top_20)))
ax.set_yticklabels(top_20.index)
ax.set_xlabel('Absolute Correlation with Returns')
ax.set_title('Top 20 Features by Correlation with Stock Returns')
ax.invert_yaxis()
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '03_feature_correlations.png'), bbox_inches='tight')
print("✓ Chart 3: Feature Correlations saved")
plt.close()

# CHART 4: Correlation Heatmap
print("\n[Creating correlation heatmap...]")
top_features = correlations.head(15).index.tolist()
corr_matrix = df[top_features + [TARGET]].corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
ax.set_title('Correlation Heatmap: Top 15 Features + Returns')
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '04_correlation_heatmap.png'), bbox_inches='tight')
print("✓ Chart 4: Correlation Heatmap saved")
plt.close()

# CHART 5: Missing Data Pattern
print("\n[Creating missing data chart...]")
df_original = pd.read_csv(DATA_PATH)
missing_counts = df_original.isnull().sum().sort_values(ascending=False).head(20)

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(range(len(missing_counts)), missing_counts.values, color='coral')
ax.set_xticks(range(len(missing_counts)))
ax.set_xticklabels(missing_counts.index, rotation=45, ha='right')
ax.set_ylabel('Number of Missing Values')
ax.set_title('Top 20 Features with Missing Values (Before Imputation)')
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '05_missing_data_pattern.png'), bbox_inches='tight')
print("✓ Chart 5: Missing Data Pattern saved")
plt.close()

# ============================================================================
# 4. TRAIN-TEST SPLIT
# ============================================================================
print("\n[STEP 4] Train-Validation-Test Split...")

df = df.sort_values('DATE').reset_index(drop=True)
unique_dates = sorted(df['DATE'].unique())
n_dates = len(unique_dates)

train_end = int(0.6 * n_dates)
val_end = int(0.8 * n_dates)

train_dates = unique_dates[:train_end]
val_dates = unique_dates[train_end:val_end]
test_dates = unique_dates[val_end:]

print(f"Train: {train_dates[0]} to {train_dates[-1]} ({len(train_dates)} months)")
print(f"Val: {val_dates[0]} to {val_dates[-1]} ({len(val_dates)} months)")
print(f"Test: {test_dates[0]} to {test_dates[-1]} ({len(test_dates)} months)")

train_df = df[df['DATE'].isin(train_dates)].copy()
val_df = df[df['DATE'].isin(val_dates)].copy()
test_df = df[df['DATE'].isin(test_dates)].copy()

X_train = train_df[FEATURE_COLS]
y_train = train_df[TARGET]
X_val = val_df[FEATURE_COLS]
y_val = val_df[TARGET]
X_test = test_df[FEATURE_COLS]
y_test = test_df[TARGET]

# Scaling
print("\n[Feature Scaling...]")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# CHART 6: Data Split Visualization
fig, ax = plt.subplots(figsize=(12, 6))
split_data = pd.DataFrame({
    'Period': ['Train', 'Validation', 'Test'],
    'Observations': [len(train_df), len(val_df), len(test_df)],
    'Months': [len(train_dates), len(val_dates), len(test_dates)]
})

x = np.arange(len(split_data))
width = 0.35

ax.bar(x - width/2, split_data['Observations'], width, label='Observations', color='steelblue')
ax2 = ax.twinx()
ax2.bar(x + width/2, split_data['Months'], width, label='Months', color='coral')

ax.set_xlabel('Dataset Split')
ax.set_ylabel('Number of Observations', color='steelblue')
ax2.set_ylabel('Number of Months', color='coral')
ax.set_title('Train-Validation-Test Split')
ax.set_xticks(x)
ax.set_xticklabels(split_data['Period'])
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '06_data_split.png'), bbox_inches='tight')
print("✓ Chart 6: Data Split Visualization saved")
plt.close()

# ============================================================================
# 5. MODEL TRAINING
# ============================================================================
print("\n" + "="*80)
print("[STEP 5] Model Training - ALL MODELS")
print("="*80)

results = {}
training_times = {}

# ---------------------------------------------------------------------------
# 5.1 LINEAR MODELS
# ---------------------------------------------------------------------------
print("\n--- LINEAR MODELS ---")

# OLS
print("\n[1] Training OLS...")
start = time.time()
lr_ols = LinearRegression()
lr_ols.fit(X_train_scaled, y_train)
training_times['OLS'] = time.time() - start
results['OLS'] = {
    'train': lr_ols.predict(X_train_scaled),
    'val': lr_ols.predict(X_val_scaled),
    'test': lr_ols.predict(X_test_scaled)
}
print(f"✓ OLS completed in {training_times['OLS']:.2f}s")

# Ridge
print("\n[2] Training Ridge...")
start = time.time()
lr_ridge = Ridge(alpha=1.0)
lr_ridge.fit(X_train_scaled, y_train)
training_times['Ridge'] = time.time() - start
results['Ridge'] = {
    'train': lr_ridge.predict(X_train_scaled),
    'val': lr_ridge.predict(X_val_scaled),
    'test': lr_ridge.predict(X_test_scaled)
}
print(f"✓ Ridge completed in {training_times['Ridge']:.2f}s")

# Lasso
print("\n[3] Training Lasso...")
start = time.time()
lr_lasso = Lasso(alpha=0.001, max_iter=5000)
lr_lasso.fit(X_train_scaled, y_train)
training_times['Lasso'] = time.time() - start
results['Lasso'] = {
    'train': lr_lasso.predict(X_train_scaled),
    'val': lr_lasso.predict(X_val_scaled),
    'test': lr_lasso.predict(X_test_scaled)
}
print(f"✓ Lasso completed in {training_times['Lasso']:.2f}s")

# ---------------------------------------------------------------------------
# 5.2 TREE-BASED MODELS
# ---------------------------------------------------------------------------
print("\n--- TREE-BASED MODELS ---")

# Random Forest
print("\n[4] Training Random Forest...")
start = time.time()
rf = RandomForestRegressor(
    n_estimators=50,
    max_depth=8,
    min_samples_split=100,
    n_jobs=-1,
    random_state=42
)
rf.fit(X_train, y_train)
training_times['RandomForest'] = time.time() - start
results['RandomForest'] = {
    'train': rf.predict(X_train),
    'val': rf.predict(X_val),
    'test': rf.predict(X_test)
}
print(f"✓ Random Forest completed in {training_times['RandomForest']:.2f}s")

# Gradient Boosting
print("\n[5] Training Gradient Boosting...")
start = time.time()
gb = GradientBoostingRegressor(
    n_estimators=50,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)
gb.fit(X_train, y_train)
training_times['GradientBoosting'] = time.time() - start
results['GradientBoosting'] = {
    'train': gb.predict(X_train),
    'val': gb.predict(X_val),
    'test': gb.predict(X_test)
}
print(f"✓ Gradient Boosting completed in {training_times['GradientBoosting']:.2f}s")

# ---------------------------------------------------------------------------
# 5.3 NEURAL NETWORK
# ---------------------------------------------------------------------------
print("\n--- NEURAL NETWORK ---")
print("\n[6] Training Neural Network...")
start = time.time()

# Build neural network
nn = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
          loss='mse', 
          metrics=['mae'])

# Early stopping
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train
history = nn.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=50,
    batch_size=512,
    callbacks=[early_stop],
    verbose=0
)

training_times['NeuralNetwork'] = time.time() - start

y_pred_nn_train = nn.predict(X_train_scaled, verbose=0).flatten()
y_pred_nn_val = nn.predict(X_val_scaled, verbose=0).flatten()
y_pred_nn_test = nn.predict(X_test_scaled, verbose=0).flatten()

results['NeuralNetwork'] = {
    'train': y_pred_nn_train,
    'val': y_pred_nn_val,
    'test': y_pred_nn_test
}
print(f"✓ Neural Network completed in {training_times['NeuralNetwork']:.2f}s")

# CHART 7: Neural Network Training History
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE')
axes[0].set_title('Neural Network Training: Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history.history['mae'], label='Train MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Val MAE', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('Neural Network Training: Mean Absolute Error')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '07_nn_training_history.png'), bbox_inches='tight')
print("✓ Chart 7: Neural Network Training History saved")
plt.close()

# CHART 8: Training Times
fig, ax = plt.subplots(figsize=(10, 6))
models_list = list(training_times.keys())
times = list(training_times.values())
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
ax.barh(models_list, times, color=colors)
ax.set_xlabel('Training Time (seconds)')
ax.set_title('Model Training Time Comparison')
ax.grid(True, alpha=0.3, axis='x')
for i, v in enumerate(times):
    ax.text(v, i, f' {v:.2f}s', va='center')
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '08_training_times.png'), bbox_inches='tight')
print("✓ Chart 8: Training Times saved")
plt.close()

# ============================================================================
# 6. PREDICTIVE PERFORMANCE EVALUATION
# ============================================================================
print("\n" + "="*80)
print("[STEP 6] Predictive Performance Evaluation")
print("="*80)

def evaluate_predictions(y_true, y_pred, dataset_name, model_name):
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    return {'Model': model_name, 'Dataset': dataset_name, 'R2': r2, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}

performance_metrics = []
for model_name, preds in results.items():
    performance_metrics.append(evaluate_predictions(y_train, preds['train'], 'Train', model_name))
    performance_metrics.append(evaluate_predictions(y_val, preds['val'], 'Validation', model_name))
    performance_metrics.append(evaluate_predictions(y_test, preds['test'], 'Test', model_name))

perf_df = pd.DataFrame(performance_metrics)
print("\n[Performance Metrics]")
print(perf_df)

perf_df.to_csv(os.path.join(OUTPUT_DIR, 'predictive_performance_detailed.csv'), index=False)
print(f"\n✓ Saved to '{OUTPUT_DIR}/predictive_performance_detailed.csv'")

# CHART 9: R-squared Comparison
fig, ax = plt.subplots(figsize=(12, 6))
test_perf = perf_df[perf_df['Dataset'] == 'Test']
x = np.arange(len(test_perf))
ax.bar(x, test_perf['R2'], color=colors)
ax.set_xlabel('Model')
ax.set_ylabel('R-squared')
ax.set_title('Model Performance: R² on Test Set (Out-of-Sample)')
ax.set_xticks(x)
ax.set_xticklabels(test_perf['Model'], rotation=15)
ax.grid(True, alpha=0.3, axis='y')
for i, v in enumerate(test_perf['R2']):
    ax.text(i, v, f'{v:.4f}', ha='center', va='bottom')
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '09_r2_comparison.png'), bbox_inches='tight')
print("✓ Chart 9: R² Comparison saved")
plt.close()

# CHART 10: MSE Comparison
fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(x, test_perf['MSE'], color=colors)
ax.set_xlabel('Model')
ax.set_ylabel('Mean Squared Error')
ax.set_title('Model Performance: MSE on Test Set (Lower is Better)')
ax.set_xticks(x)
ax.set_xticklabels(test_perf['Model'], rotation=15)
ax.grid(True, alpha=0.3, axis='y')
for i, v in enumerate(test_perf['MSE']):
    ax.text(i, v, f'{v:.6f}', ha='center', va='bottom', fontsize=8)
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '10_mse_comparison.png'), bbox_inches='tight')
print("✓ Chart 10: MSE Comparison saved")
plt.close()

# CHART 11: Actual vs Predicted (Best Model)
best_model = test_perf.loc[test_perf['R2'].idxmax(), 'Model']
print(f"\nBest model by R²: {best_model}")

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].scatter(y_test, results[best_model]['test'], alpha=0.3, s=10, color='steelblue')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Returns')
axes[0].set_ylabel('Predicted Returns')
axes[0].set_title(f'Actual vs Predicted Returns: {best_model}')
axes[0].grid(True, alpha=0.3)

residuals = y_test - results[best_model]['test']
axes[1].scatter(results[best_model]['test'], residuals, alpha=0.3, s=10, color='coral')
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Predicted Returns')
axes[1].set_ylabel('Residuals')
axes[1].set_title(f'Residual Plot: {best_model}')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '11_actual_vs_predicted.png'), bbox_inches='tight')
print("✓ Chart 11: Actual vs Predicted saved")
plt.close()

# ============================================================================
# 7. PORTFOLIO PERFORMANCE
# ============================================================================
print("\n" + "="*80)
print("[STEP 7] Portfolio Performance Evaluation")
print("="*80)

def portfolio_performance(df_subset, predictions, dates_list):
    df_temp = df_subset.copy()
    df_temp['Prediction'] = predictions
    monthly_returns = []
    
    for date in dates_list:
        date_data = df_temp[df_temp['DATE'] == date]
        if len(date_data) < 100:
            continue
        top100 = date_data.nlargest(100, 'Prediction')
        monthly_returns.append(top100[TARGET].mean())
    
    return {
        'Avg_Return': np.mean(monthly_returns),
        'Volatility': np.std(monthly_returns),
        'Sharpe_Ratio': np.mean(monthly_returns) / np.std(monthly_returns) if np.std(monthly_returns) > 0 else 0,
        'N_Months': len(monthly_returns),
        'Returns': monthly_returns
    }

portfolio_metrics = []
portfolio_returns_dict = {}

for model_name, preds in results.items():
    port_train = portfolio_performance(train_df, preds['train'], train_dates)
    port_test = portfolio_performance(test_df, preds['test'], test_dates)
    
    portfolio_metrics.append({
        'Model': model_name, 
        'Dataset': 'Train',
        'Avg_Return': port_train['Avg_Return'],
        'Volatility': port_train['Volatility'],
        'Sharpe_Ratio': port_train['Sharpe_Ratio']
    })
    portfolio_metrics.append({
        'Model': model_name,
        'Dataset': 'Test',
        'Avg_Return': port_test['Avg_Return'],
        'Volatility': port_test['Volatility'],
        'Sharpe_Ratio': port_test['Sharpe_Ratio']
    })
    
    portfolio_returns_dict[model_name] = port_test['Returns']

port_df = pd.DataFrame(portfolio_metrics)
print("\n[Portfolio Performance]")
print(port_df)

port_df.to_csv(os.path.join(OUTPUT_DIR, 'portfolio_performance.csv'), index=False)
print(f"\n✓ Saved to '{OUTPUT_DIR}/portfolio_performance.csv'")

# CHART 12: Portfolio Metrics
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
port_test = port_df[port_df['Dataset'] == 'Test']

axes[0].barh(port_test['Model'], port_test['Avg_Return'], color=colors)
axes[0].set_xlabel('Average Monthly Return')
axes[0].set_title('Portfolio Average Return (Test Set)')
axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[0].grid(True, alpha=0.3, axis='x')

axes[1].barh(port_test['Model'], port_test['Volatility'], color=colors)
axes[1].set_xlabel('Volatility (Std Dev)')
axes[1].set_title('Portfolio Volatility (Test Set)')
axes[1].grid(True, alpha=0.3, axis='x')

axes[2].barh(port_test['Model'], port_test['Sharpe_Ratio'], color=colors)
axes[2].set_xlabel('Sharpe Ratio')
axes[2].set_title('Portfolio Sharpe Ratio (Test Set)')
axes[2].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[2].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '12_portfolio_metrics.png'), bbox_inches='tight')
print("✓ Chart 12: Portfolio Metrics saved")
plt.close()

# CHART 13: Cumulative Returns
fig, ax = plt.subplots(figsize=(14, 7))
for model_name, returns in portfolio_returns_dict.items():
    cumulative = np.cumprod(1 + np.array(returns)) - 1
    ax.plot(range(len(cumulative)), cumulative, label=model_name, linewidth=2)

ax.set_xlabel('Months (Test Period)')
ax.set_ylabel('Cumulative Return')
ax.set_title('Cumulative Portfolio Returns Over Time (Test Period)')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '13_cumulative_returns.png'), bbox_inches='tight')
print("✓ Chart 13: Cumulative Returns saved")
plt.close()

# ============================================================================
# 8. FEATURE IMPORTANCE
# ============================================================================
print("\n" + "="*80)
print("[STEP 8] Feature Importance Analysis")
print("="*80)

rf_importance = pd.DataFrame({
    'Feature': FEATURE_COLS,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 20 Important Features:")
print(rf_importance.head(20))

rf_importance.to_csv(os.path.join(OUTPUT_DIR, 'feature_importance.csv'), index=False)
print(f"\n✓ Saved to '{OUTPUT_DIR}/feature_importance.csv'")

# CHART 14: Feature Importance
fig, ax = plt.subplots(figsize=(10, 8))
top_20_imp = rf_importance.head(20)
ax.barh(range(len(top_20_imp)), top_20_imp['Importance'], color='forestgreen')
ax.set_yticks(range(len(top_20_imp)))
ax.set_yticklabels(top_20_imp['Feature'])
ax.set_xlabel('Importance Score')
ax.set_title('Top 20 Most Important Features (Random Forest)')
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '14_feature_importance.png'), bbox_inches='tight')
print("✓ Chart 14: Feature Importance saved")
plt.close()

# ============================================================================
# 9. MODEL COMPARISON SUMMARY
# ============================================================================
print("\n" + "="*80)
print("[STEP 9] Creating Summary Charts")
print("="*80)

# CHART 15: Train vs Test Performance
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

train_perf = perf_df[perf_df['Dataset'] == 'Train']
test_perf_chart = perf_df[perf_df['Dataset'] == 'Test']

x = np.arange(len(train_perf))
width = 0.35

axes[0].bar(x - width/2, train_perf['R2'], width, label='Train', color='steelblue', alpha=0.8)
axes[0].bar(x + width/2, test_perf_chart['R2'], width, label='Test', color='coral', alpha=0.8)
axes[0].set_xlabel('Model')
axes[0].set_ylabel('R-squared')
axes[0].set_title('R² Comparison: Train vs Test (Overfitting Check)')
axes[0].set_xticks(x)
axes[0].set_xticklabels(train_perf['Model'], rotation=15, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].bar(x - width/2, train_perf['MSE'], width, label='Train', color='steelblue', alpha=0.8)
axes[1].bar(x + width/2, test_perf_chart['MSE'], width, label='Test', color='coral', alpha=0.8)
axes[1].set_xlabel('Model')
axes[1].set_ylabel('MSE')
axes[1].set_title('MSE Comparison: Train vs Test')
axes[1].set_xticks(x)
axes[1].set_xticklabels(train_perf['Model'], rotation=15, ha='right')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '15_train_vs_test.png'), bbox_inches='tight')
print("✓ Chart 15: Train vs Test Comparison saved")
plt.close()

# CHART 16: Risk-Return Tradeoff
fig, ax = plt.subplots(figsize=(10, 8))
port_test_plot = port_df[port_df['Dataset'] == 'Test']

colors_map = {'OLS': '#1f77b4', 'Ridge': '#ff7f0e', 'Lasso': '#2ca02c', 
              'RandomForest': '#d62728', 'GradientBoosting': '#9467bd',
              'NeuralNetwork': '#8c564b'}

for idx, row in port_test_plot.iterrows():
    ax.scatter(row['Volatility'], row['Avg_Return'], 
              s=300, alpha=0.7, color=colors_map[row['Model']], 
              label=row['Model'], edgecolors='black', linewidth=1.5)

ax.set_xlabel('Portfolio Volatility (Risk)')
ax.set_ylabel('Average Return')
ax.set_title('Risk-Return Tradeoff: Portfolio Performance')
ax.grid(True, alpha=0.3)
ax.legend()
ax.axhline(y=0, color='red', linestyle='--', linewidth=0.8, alpha=0.5)
plt.tight_layout()
plt.savefig(os.path.join(CHARTS_DIR, '16_risk_return_tradeoff.png'), bbox_inches='tight')
print("✓ Chart 16: Risk-Return Tradeoff saved")
plt.close()

# ============================================================================
# 10. FINAL SUMMARY
# ============================================================================
print("\n" + "="*80)
print("PROJECT COMPLETE!")
print("="*80)

print(f"\n📊 RESULTS SUMMARY:")
print(f"   Dataset: {df.shape[0]:,} observations, {len(FEATURE_COLS)} features")
print(f"   Models trained: {len(results)} (Linear + Tree-Based + Neural Network)")
print(f"   Test period: {len(test_dates)} months")

print(f"\n📁 FILES SAVED TO: {OUTPUT_DIR}")
print("   CSV Files:")
print("   ✓ predictive_performance_detailed.csv")
print("   ✓ portfolio_performance.csv")
print("   ✓ feature_importance.csv")

print(f"\n📈 CHARTS SAVED TO: {CHARTS_DIR}")
charts_list = [
    "01_target_distribution.png",
    "02_returns_over_time.png",
    "03_feature_correlations.png",
    "04_correlation_heatmap.png",
    "05_missing_data_pattern.png",
    "06_data_split.png",
    "07_nn_training_history.png",
    "08_training_times.png",
    "09_r2_comparison.png",
    "10_mse_comparison.png",
    "11_actual_vs_predicted.png",
    "12_portfolio_metrics.png",
    "13_cumulative_returns.png",
    "14_feature_importance.png",
    "15_train_vs_test.png",
    "16_risk_return_tradeoff.png"
]

for i, chart in enumerate(charts_list, 1):
    print(f"   ✓ {chart}")

print(f"\n🏆 BEST MODEL (by R²): {best_model}")
best_r2 = test_perf.loc[test_perf['R2'].idxmax(), 'R2']
print(f"   Test R²: {best_r2:.6f}")

best_sharpe_idx = port_df[port_df['Dataset'] == 'Test']['Sharpe_Ratio'].idxmax()
best_sharpe_model = port_df.loc[best_sharpe_idx, 'Model']
best_sharpe = port_df.loc[best_sharpe_idx, 'Sharpe_Ratio']
print(f"\n🏆 BEST PORTFOLIO (by Sharpe Ratio): {best_sharpe_model}")
print(f"   Sharpe Ratio: {best_sharpe:.4f}")


FA 590 - Stock Return Prediction Project - FULL VERSION

[STEP 1] Loading Data...
Data location: /Users/harshil/Desktop/Statistical Learning Final Project/return_predictability_data.csv
Outputs will be saved to: /Users/harshil/Desktop/Statistical Learning Final Project
Charts will be saved to: /Users/harshil/Desktop/Statistical Learning Final Project/charts

Dataset Shape: (4089924, 107)
Dataset Size: 4043.75 MB in memory
Date Range: 1957-01-31 to 2021-11-30
Number of unique stocks: 32655
Number of unique dates: 779

[STEP 2] Data Preprocessing...
Number of features: 103

[Converting data types...]
Dataset shape after preprocessing: (4089903, 107)

[Encoding categorical variable: sic2]
Features after encoding: 176

[STEP 3] Exploratory Data Analysis & Creating Charts...
✓ Chart 1: Target Distribution saved

[Creating time series chart...]
✓ Chart 2: Returns Over Time saved

[Calculating correlations...]
✓ Chart 3: Feature Correlations saved

[Creating correlation heatmap...]
✓ Chart 4: